# Proyecto Clasificación de hojas de aguacate

---



## Instalación de paquetes e importación de líbrerias
---

En esta sección se encuentra el procedimiento para instalar Detectron2. En caso de presentar dificultades al correr la librería se recomienda actualizar las versiones de las subsecciones:
- *Instalación de torch*
- *Instalación de Detectron*

**Nota:** Al correr cada una de las subsecciones *Instalación de torch* e *Instalación de Detectron* recuerde reiniciar el entorno de ejecución.


In [ ]:
#@title Líbrerias Estandar

import matplotlib.pyplot as plt
import numpy as np
import os, PIL, cv2, json, random
import tensorflow as tf
import matplotlib.pyplot as plt

!pip install pyyaml==5.1

In [ ]:
#@title Importación de los datos
from google.colab import drive
drive.mount('/content/drive')

import pathlib
data_dir_str_1 = "/content/drive/MyDrive/Marchitez"
data_dir_str ='/content/drive/Shareddrives/Proyecto Avocado/DatasetAvocado'
data_dir = pathlib.Path(data_dir_str)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#@title Instalación de torch versión 1.8.0
!pip install torch==1.8.0+cu101 torchvision==0.9.0+cu101 -f https://download.pytorch.org/whl/torch_stable.html
#install old version of pytorch since detectron2 hasn't released packages for pyton

In [ ]:
#@title Instalación de Detectron
!pip install detectron2 -f https://dl.fbaipublicfiles.com/detectron2/wheels/cu101/torch1.8/index.html
# After this step it will ask you to restart the runtime, please do it.

In [ ]:
#@title Importación de las dependencias de Detectron2
from detectron2.structures import BoxMode
from detectron2.data import  DatasetCatalog, MetadataCatalog

from detectron2 import model_zoo
from detectron2.engine import DefaultTrainer, DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import ColorMode, Visualizer

## Registrar los datos en la configuración de Detectron2
---


In [ ]:
#@title Convert Json format to Coco format

def get_data_dicts(directory, classes):
    dataset_dicts = []
    for filename in [file for file in os.listdir(directory) if file.endswith('.json')]:
        json_file = os.path.join(directory, filename)
        with open(json_file) as f:
            img_anns = json.load(f)

        record = {}
        
        filename = os.path.join(directory, img_anns["imagePath"])
        
        record["file_name"] = filename
        record["height"] = 1080#224
        record["width"] = 1440 #224
      
        annos = img_anns["shapes"]
        objs = []
        for anno in annos:
            px = [a[0] for a in anno['points']] # x coord
            py = [a[1] for a in anno['points']] # y-coord
            poly = [(x, y) for x, y in zip(px, py)] # poly for segmentation
            poly = [p for x in poly for p in x]

            obj = {
                "bbox": [np.min(px), np.min(py), np.max(px), np.max(py)],
                "bbox_mode": BoxMode.XYXY_ABS,
                "segmentation": [poly],
                "category_id": classes.index(anno['label']),
                "iscrowd": 0
            }
            objs.append(obj)
        record["annotations"] = objs
        dataset_dicts.append(record)
    return dataset_dicts

In [ ]:
classes = ['Leaf']

data_path = '/content/drive/Shareddrives/Proyecto Avocado/Detectron2_Data_set/'

for d in ["train", "test"]:
    DatasetCatalog.register(
        "category_" + d, 
        lambda d=d: get_data_dicts(data_path+d, classes)
    )
    MetadataCatalog.get("category_" + d).set(thing_classes=classes)

leaf_metadata = MetadataCatalog.get("category_train")


## Entrenando el modelo "Instance Segmentation" de Detectron2

In [ ]:
cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))
cfg.DATASETS.TRAIN = ("category_train",)
cfg.DATASETS.TEST = ()
cfg.DATALOADER.NUM_WORKERS = 2
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml")
cfg.SOLVER.IMS_PER_BATCH = 2
cfg.SOLVER.BASE_LR = 0.00025
cfg.SOLVER.MAX_ITER = 1000
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 1

In [ ]:
os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
trainer = DefaultTrainer(cfg) 
trainer.resume_or_load(resume=False)

[04/26 00:30:01 d2.engine.defaults]: Model:
GeneralizedRCNN(
  (backbone): FPN(
    (fpn_lateral2): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral3): Conv2d(512, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral4): Conv2d(1024, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output4): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral5): Conv2d(2048, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output5): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (top_block): LastLevelMaxPool()
    (bottom_up): ResNet(
      (stem): BasicStem(
        (conv1): Conv2d(
          3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False
          (norm): FrozenBatchNorm2d(num_features=64, eps=1e-05)
        )
      )
      (res

Skip loading parameter 'roi_heads.box_predictor.cls_score.weight' to the model due to incompatible shapes: (81, 1024) in the checkpoint but (2, 1024) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.cls_score.bias' to the model due to incompatible shapes: (81,) in the checkpoint but (2,) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.bbox_pred.weight' to the model due to incompatible shapes: (320, 1024) in the checkpoint but (4, 1024) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.bbox_pred.bias' to the model due to incompatible shapes: (320,) in the checkpoint but (4,) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.mask_head.predictor.weight' to the model due to incompatible shapes: (80, 256, 1, 1) in the checkpoint but (1, 256, 1, 1) in

In [ ]:
trainer.train()

[04/26 00:30:01 d2.engine.train_loop]: Starting training from iteration 0
[04/26 00:30:30 d2.utils.events]:  eta: 0:22:44  iter: 19  total_loss: 2.245  loss_cls: 0.6832  loss_box_reg: 0.3138  loss_mask: 0.6931  loss_rpn_cls: 0.5533  loss_rpn_loc: 0.06769  time: 1.3824  data_time: 0.0498  lr: 4.9953e-06  max_mem: 2782M
[04/26 00:30:57 d2.utils.events]:  eta: 0:22:25  iter: 39  total_loss: 2.285  loss_cls: 0.6283  loss_box_reg: 0.4046  loss_mask: 0.6886  loss_rpn_cls: 0.4778  loss_rpn_loc: 0.06471  time: 1.3830  data_time: 0.0078  lr: 9.9902e-06  max_mem: 2782M
[04/26 00:31:26 d2.utils.events]:  eta: 0:21:53  iter: 59  total_loss: 2.037  loss_cls: 0.5237  loss_box_reg: 0.4712  loss_mask: 0.6764  loss_rpn_cls: 0.2216  loss_rpn_loc: 0.05461  time: 1.3936  data_time: 0.0086  lr: 1.4985e-05  max_mem: 2902M
[04/26 00:31:54 d2.utils.events]:  eta: 0:21:25  iter: 79  total_loss: 1.771  loss_cls: 0.4556  loss_box_reg: 0.5041  loss_mask: 0.6578  loss_rpn_cls: 0.1036  loss_rpn_loc: 0.04753  time: 

## Inference using the Trained Model

In [ ]:
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5 
cfg.DATASETS.TEST = ("skin_test", )
predictor = DefaultPredictor(cfg)

In [ ]:
test_dataset_dicts = get_data_dicts(data_path+'test', classes)

In [ ]:
for d in random.sample(test_dataset_dicts, 5):    
    img = cv2.imread(d["file_name"])
    outputs = predictor(img)
    v = Visualizer(img[:, :, ::-1],
                   metadata=leaf_metadata, 
                   scale=0.8, 
                   instance_mode=ColorMode.IMAGE_BW # removes the colors of unsegmented pixels
    )
    v = v.draw_instance_predictions(outputs["instances"].to("cpu"))
    plt.figure(figsize = (10, 8))
    plt.imshow(cv2.cvtColor(v.get_image()[:, :, ::-1], cv2.COLOR_BGR2RGB))
    plt.show()

Output hidden; open in https://colab.research.google.com to view.